# CLI Interface

> Command-line interface for docments compliance checking

In [ ]:
#| default_exp cli

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import argparse
import sys
import json
from pathlib import Path
from typing import Optional
from cjm_nbdev_docments.report import check_project, generate_text_report, generate_json_report
from nbdev.config import get_config

In [ ]:
#| export
def create_parser(
) -> argparse.ArgumentParser:  # Configured argument parser
    "Create and configure the argument parser for docments CLI"
    parser = argparse.ArgumentParser(
        description="Check nbdev project for docments compliance",
        formatter_class=argparse.RawDescriptionHelpFormatter,
        epilog="""
Examples:
  # Check current project
  nbdev-docments
  
  # Check specific notebooks directory
  nbdev-docments --nbs-path ./notebooks
  
  # Generate JSON report
  nbdev-docments --format json
  
  # Save report to file
  nbdev-docments --output report.txt
  
  # Show all definitions (including compliant ones)
  nbdev-docments --verbose
  
  # Show only functions with TODO placeholders
  nbdev-docments --todos-only
  
  # Auto-fix non-compliant functions
  nbdev-docments --fix
  
  # Auto-fix with docstring conversion
  nbdev-docments --fix --convert-docstrings
  
  # Preview fixes without applying
  nbdev-docments --fix --dry-run
"""
    )
    
    parser.add_argument(
        "--nbs-path",
        type=Path,
        help="Path to notebooks directory (defaults to nbdev config)"
    )
    
    parser.add_argument(
        "--format",
        choices=["text", "json"],
        default="text",
        help="Output format (default: text)"
    )
    
    parser.add_argument(
        "--output", "-o",
        type=Path,
        help="Save report to file instead of printing"
    )
    
    parser.add_argument(
        "--verbose", "-v",
        action="store_true",
        help="Show compliant definitions in text report"
    )
    
    parser.add_argument(
        "--quiet", "-q",
        action="store_true",
        help="Only show summary (exit code indicates compliance)"
    )
    
    parser.add_argument(
        "--todos-only",
        action="store_true",
        help="Show only functions with TODO placeholders"
    )
    
    parser.add_argument(
        "--fix",
        action="store_true",
        help="Auto-fix non-compliant functions by adding placeholder docs"
    )
    
    parser.add_argument(
        "--convert-docstrings",
        action="store_true",
        help="Convert existing Google/NumPy/Sphinx docstrings to docments format (use with --fix)"
    )
    
    parser.add_argument(
        "--dry-run",
        action="store_true",
        help="Show what would be fixed without making changes"
    )
    
    return parser

In [ ]:
#| export
def handle_autofix(
    args: argparse.Namespace  # Parsed command line arguments
) -> int:  # Exit code
    "Handle auto-fix mode for non-compliant functions"
    from cjm_nbdev_docments.autofix import fix_notebook, fix_notebook_with_conversion
    
    if args.nbs_path:
        nbs_path = args.nbs_path
    else:
        cfg = get_config()
        nbs_path = Path(cfg.config_path) / cfg.nbs_path
    
    # Choose the appropriate fix function based on whether conversion is requested
    if args.convert_docstrings:
        fix_function = lambda nb_path, dry_run: fix_notebook_with_conversion(
            nb_path, dry_run=dry_run, convert_docstrings=True
        )
    else:
        fix_function = fix_notebook
    
    # Fix all notebooks (including in subdirectories)
    total_fixed = 0
    for nb_path in nbs_path.rglob("*.ipynb"):
        # Skip private notebooks and those in .ipynb_checkpoints
        if not nb_path.name.startswith('_') and '.ipynb_checkpoints' not in str(nb_path):
            try:
                changes = fix_function(nb_path, args.dry_run)
                total_fixed += len(changes['definitions_fixed'])
            except Exception as e:
                print(f"Error fixing {nb_path}: {e}", file=sys.stderr)
    
    return 0 if not args.dry_run else (1 if total_fixed > 0 else 0)

In [ ]:
#| export
def generate_report(
    results: list,  # Check results from check_project
    format: str,  # Output format ("text" or "json")
    verbose: bool = False  # Whether to show compliant definitions
) -> str:  # Generated report as string
    "Generate a report in the specified format"
    if format == "json":
        return json.dumps(generate_json_report(results), indent=2)
    else:
        return generate_text_report(results, verbose=verbose)

In [ ]:
#| export
def output_report(
    report: str,  # Report content to output
    output_path: Optional[Path] = None,  # File path to save report to
    quiet: bool = False  # Whether to suppress output
) -> None:
    "Output the report to console or file"
    if not quiet:
        if output_path:
            output_path.write_text(report)
            print(f"Report saved to {output_path}")
        else:
            print(report)

In [ ]:
#| export
def main(
    args: Optional[list] = None  # Command line arguments (for testing)
) -> int:  # Exit code (0 for success, 1 for non-compliance)
    "Main CLI entry point for docments checker"
    parser = create_parser()
    parsed_args = parser.parse_args(args)
    
    # Handle auto-fix mode
    if parsed_args.fix:
        return handle_autofix(parsed_args)
    
    # Normal check mode
    try:
        results = check_project(parsed_args.nbs_path)
    except Exception as e:
        print(f"Error: {e}", file=sys.stderr)
        return 1
    
    # Filter for TODOs only if requested
    if parsed_args.todos_only:
        results = [r for r in results if r.has_todos]
        if not results:
            print("✅ No functions with TODO placeholders found!")
            return 0
    
    # Generate and output report
    report = generate_report(results, parsed_args.format, parsed_args.verbose)
    output_report(report, parsed_args.output, parsed_args.quiet)
    
    # Return exit code based on compliance
    non_compliant = sum(1 for r in results if not r.is_compliant)
    return 0 if non_compliant == 0 else 1

In [ ]:
# Test the CLI with different options
print("Basic check:")
exit_code = main([])
print(f"\nExit code: {exit_code}")

print("\n" + "="*50 + "\n")

print("JSON format:")
main(["--format", "json"])

Basic check:
📚 Docments Compliance Report
Total definitions: 58
✅ Compliant: 58
❌ Non-compliant: 0
⚠️  With TODO placeholders: 5


⚠️  Definitions with TODO placeholders:
------------------------------

📓 03_autofix.ipynb:
  ⚠️  needs_fixing (1 TODOs)
  ⚠️  get_param_name (2 TODOs)
  ⚠️  needs_param_fix (2 TODOs)

📓 04_cli.ipynb:
  ⚠️  create_parser (1 TODOs)
  ⚠️  output_report (1 TODOs)

Exit code: 0


JSON format:
{
  "summary": {
    "total": 58,
    "compliant": 58,
    "non_compliant": 0,
    "with_todos": 5,
    "total_todos": 7
  },
  "by_notebook": {
    "00_core.ipynb": {
      "compliant": [
        {
          "name": "DocmentsCheckResult",
          "type": "ClassDef",
          "has_docstring": true,
          "missing_params": [],
          "params_documented": {},
          "return_documented": true,
          "has_todos": false,
          "todo_count": 0,
          "params_with_type_hints": {},
          "return_has_type_hint": false,
          "params_missing_type_hin

0

In [ ]:
# Test the new --convert-docstrings flag
print("Testing CLI with conversion support:")

# Test the parser with the new flag
parser = create_parser()
test_args = parser.parse_args(["--fix", "--convert-docstrings", "--dry-run"])

print(f"Parsed args:")
print(f"  fix: {test_args.fix}")
print(f"  convert_docstrings: {test_args.convert_docstrings}")
print(f"  dry_run: {test_args.dry_run}")

print("\nCLI help text for the new flag:")
parser.print_help()

Testing CLI with conversion support:
Parsed args:
  fix: True
  convert_docstrings: True
  dry_run: True

CLI help text for the new flag:
usage: ipykernel_launcher.py [-h] [--nbs-path NBS_PATH] [--format {text,json}]
                             [--output OUTPUT] [--verbose] [--quiet]
                             [--todos-only] [--fix] [--convert-docstrings]
                             [--dry-run]

Check nbdev project for docments compliance

options:
  -h, --help            show this help message and exit
  --nbs-path NBS_PATH   Path to notebooks directory (defaults to nbdev config)
  --format {text,json}  Output format (default: text)
  --output OUTPUT, -o OUTPUT
                        Save report to file instead of printing
  --verbose, -v         Show compliant definitions in text report
  --quiet, -q           Only show summary (exit code indicates compliance)
  --todos-only          Show only functions with TODO placeholders
  --fix                 Auto-fix non-compliant functi

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()